In [3]:
import re
import csv
import string
import operator

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

import pandas as pd
import numpy as np

import requests
import json

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/catarina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def generate_ngrams(sentence: str, n: int):
    # Convert sentence to lowercase and remove punctuation
    import string
    sentence = sentence.lower().translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()
        
    # Generate N-grams
    ngrams = []
    for i in range(len(words) - n + 1):
        ngrams.append(' '.join(words[i:i+n]))

    return ngrams

In [5]:
def build_dict(seasons, characters,n):
    values = []
    
    for character in characters:
            lines = []
            counts = []
            words = []

            for season in seasons:
                for episode in season['episodes']:
                    for scene in episode['scenes']:
                        for utterance in scene['utterances']:
                            if utterance['speakers'] != []:
                                if character in utterance['speakers']:
                                    lines.append(utterance['transcript'])

            n_grams = []

            for line in lines:
                #remve punctuation
                line = line.translate(str.maketrans('', '', string.punctuation))

                if n==1:
                    line = word_tokenize(line)

                    for word in line:
                        if word not in stop_words and word != "’":
                            words.append(word.lower())    
                
                else:
                    n_gram = generate_ngrams(line,n)
                    for word in n_gram:
                        n_grams.append(word)


            if n==1:
                counts = Counter(words)
                counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
            
            else:
                counter = {}
                for elem in n_grams:
                    if elem in counter:
                        counter[elem]+=1
                    else:
                        counter[elem]=1
                counter_sorted = sorted(counter.items(), key=operator.itemgetter(1))
                counts = counter_sorted[::-1]


            values.append(counts)

    dict = {key: value for key, value in zip(characters, values)}

    return dict

In [6]:
seasons = []

for i in range(1,11):

    # load season i
    if i<10:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_0'+str(i)+'.json'
    else:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_10.json'
    
    r = requests.get(json_file)
    seasons.append(json.loads(r.text))

all_lines = []
for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if utterance['speakers'] != []:
                    all_lines.append(utterance['transcript'])

#we can do this manually
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']


In [57]:
monica_lines = []
joey_lines = []
chandler_lines = []
phoebe_lines = []
ross_lines = []
rachel_lines = []

for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if utterance['speakers'] != []:
                    if "Monica Geller" in utterance['speakers']:
                        monica_lines.append(utterance['transcript'])
                    if "Joey Tribbiani" in utterance['speakers']:
                        joey_lines.append(utterance['transcript'])
                    if "Chandler Bing" in utterance['speakers']:
                        chandler_lines.append(utterance['transcript'])
                    if "Phoebe Buffay" in utterance['speakers']:
                        phoebe_lines.append(utterance['transcript'])
                    if "Ross Geller" in utterance['speakers']:
                        ross_lines.append(utterance['transcript'])
                    if "Rachel Green" in utterance['speakers']:
                        rachel_lines.append(utterance['transcript'])
                        



In [21]:
#change 3rd argument to 1 for unigrams, 2 for bigrams, 3 for trigrams etc
dict = build_dict(seasons, characters, 4)

df = pd.DataFrame.from_dict(dict, orient='index')

df_new = df.iloc[:, 0:15]

# displaying the DataFrame
df_new.T.style

,Monica Geller,Joey Tribbiani,Chandler Bing,Phoebe Buffay,Ross Geller,Rachel Green
0,"('what are you doing', 46)","('what are you doing', 25)","('no no no no', 41)","('i know i know', 20)","('what are you doing', 38)","('i dont know i', 33)"
1,"('no no no no', 28)","('days of our lives', 25)","('what are you doing', 27)","('i dont know i', 18)","('i dont know what', 21)","('i know i know', 33)"
2,"('are you talking about', 22)","('no no no no', 24)","('you dont have to', 18)","('what are you doing', 16)","('i dont want to', 19)","('what are you doing', 32)"
3,"('what are you talking', 18)","('i dont want to', 24)","('i dont know what', 17)","('la la la la', 16)","('i dont i dont', 16)","('oh my god oh', 31)"
4,"('god oh my god', 17)","('all right all right', 20)","('i dont want to', 13)","('i dont want to', 16)","('i dont know i', 15)","('my god oh my', 25)"
5,"('my god oh my', 17)","('you know what i', 15)","('i want you to', 12)","('oh my god i', 15)","('i know i know', 15)","('god oh my god', 24)"
6,"('oh my god oh', 17)","('i know i know', 14)","('you want me to', 11)","('you dont have to', 15)","('i cant believe this', 15)","('you know what i', 22)"
7,"('what do you think', 17)","('i want you to', 13)","('oh no no no', 9)","('do you want to', 13)","('you know what i', 15)","('no no no no', 21)"
8,"('you dont have to', 17)","('to talk to you', 13)","('ow ow ow ow', 9)","('no no no no', 13)","('what do you think', 14)","('oh my god i', 20)"
9,"('why dont you just', 15)","('i dont know i', 13)","('i think we should', 8)","('you know what i', 13)","('you know what you', 14)","('i dont want to', 20)"


n-grams generics

In [27]:
def build_dict_generic(seasons,n):      
    counts = []
    words = []


    n_grams = []

    for line in all_lines:
        #remve punctuation
        line = line.translate(str.maketrans('', '', string.punctuation))

        if n==1:
            line = word_tokenize(line)

            for word in line:
                if word not in stop_words and word != "’":
                    words.append(word.lower())    
                
        else:
            n_gram = generate_ngrams(line,n)
            for word in n_gram:
                n_grams.append(word)


    if n==1:
        counts = Counter(words)
        counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
            
    else:
        counter = {}
        for elem in n_grams:
            if elem in counter:
                counter[elem]+=1
            else:
                counter[elem]=1
        counter_sorted = sorted(counter.items(), key=operator.itemgetter(1))
        counts = counter_sorted[::-1]

    return counts

In [32]:
#change 3rd argument to 1 for unigrams, 2 for bigrams, 3 for trigrams etc
counts = build_dict_generic(seasons, 5)

df = pd.DataFrame(counts, columns = ['word', 'count'])

df_new = df.iloc[:20]

# displaying the DataFrame
df_new.style

,word,count
0,no no no no no,66
1,what are you doing here,62
2,what are you talking about,57
3,my god oh my god,54
4,oh my god oh my,52
5,do you want me to,38
6,i dont know what to,34
7,can i talk to you,33
8,to talk to you about,32
9,what are you gonna do,31


In [42]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

lines_by_character = {character: [] for character in characters}

for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if utterance['speakers'] != []:
                    for speaker in utterance['speakers']:
                        if speaker in characters:
                            lines_by_character[speaker].append(utterance['transcript'])

# Create a DataFrame containing the character-specific n-grams
ngrams_by_character = {}

for character in characters:
    # Convert the character's lines into a matrix of TF-IDF scores
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
    X = vectorizer.fit_transform(lines_by_character[character])
    # Convert the matrix into a DataFrame and label the columns with the n-grams
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    # Calculate the mean TF-IDF score for each n-gram across all of the character's lines
    mean_tfidf_df = pd.DataFrame(tfidf_df.mean(axis=0), columns=['mean_tfidf_score'])
    # Remove n-grams that are also common among the other characters
    tfidf_df_other = pd.concat([pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()) for other_character in characters if other_character != character], axis=0)
    mean_tfidf_df = mean_tfidf_df.drop(columns=tfidf_df_other.columns, errors='ignore')
    # Sort the n-grams by their mean TF-IDF score and keep only the top 10 most character-specific n-grams
    character_specific_ngrams = mean_tfidf_df.sort_values('mean_tfidf_score', ascending=False).head(10)
    ngrams_by_character[character] = character_specific_ngrams

# Convert the dictionary into a single DataFrame
result_df = pd.concat(ngrams_by_character, axis=1)
result_df.columns.names = ['Character', 'N-gram']

result_df.style


Character,Monica Geller,Joey Tribbiani,Chandler Bing,Phoebe Buffay,Ross Geller,Rachel Green
N-gram,mean_tfidf_score,mean_tfidf_score,mean_tfidf_score,mean_tfidf_score,mean_tfidf_score,mean_tfidf_score
you,0.056524,0.050915,0.050523,0.054497,0.051138,0.052314
what,0.038414,0.034056,0.033335,0.031392,0.040802,0.044246
it,0.035085,0.034172,0.033797,0.034604,0.033607,0.031570
the,0.033289,0.034224,0.036335,0.031446,0.032693,0.027673
that,0.029591,0.029443,0.032132,0.029122,0.028249,0.030775
oh,0.029343,0.025423,nan,0.043714,0.026198,0.044936
to,0.026531,0.026339,0.027659,nan,0.027042,0.025975
hey,0.025297,0.048687,0.033730,0.033275,0.035439,nan
okay,0.024671,nan,0.027627,0.028614,nan,0.027538


Tabela de n-grams com tf-idf aplicado 

In [56]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Loop through each character and identify their character-specific n-grams
for character in characters:
    # Filter the DataFrame to only include the character's lines
    character_lines = []

    for season in seasons:
        for episode in season['episodes']:
            for scene in episode['scenes']:
                for utterance in scene['utterances']:
                    if utterance['speakers'] != []:
                        if character in utterance['speakers']:
                            character_lines.append(utterance['transcript'])


    # Create a TfidfVectorizer object to convert the text into a matrix of TF-IDF scores
    vectorizer = TfidfVectorizer(ngram_range=(4, 4), max_features=1000)
    X = vectorizer.fit_transform(character_lines)

    # Convert the matrix into a DataFrame and label the columns with the n-grams
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    # Calculate the mean TF-IDF score for each n-gram across all of the character's lines
    mean_tfidf_df = pd.DataFrame(tfidf_df.mean(axis=0), columns=['mean_tfidf_score'])

    # Sort the n-grams by their mean TF-IDF score and keep only the top 10 most character-specific n-grams
    character_specific_ngrams = mean_tfidf_df.sort_values('mean_tfidf_score', ascending=False).head(10)

    print(f'{character} specific n-grams:')
    print(character_specific_ngrams)

Monica Geller specific n-grams:
                       mean_tfidf_score
what are you doing             0.004477
are you talking about          0.001861
you don have to                0.001551
what are you talking           0.001540
why don you just               0.001484
oh my god you                  0.001359
what do you think              0.001345
oh my god that                 0.001242
do you want to                 0.001132
no no no no                    0.001124
Joey Tribbiani specific n-grams:
                     mean_tfidf_score
no no no no                  0.003582
what are you doing           0.002219
days of our lives            0.002029
oh no no no                  0.001993
all right all right          0.001483
what do you think            0.001044
what do you say              0.000995
you don have to              0.000972
you want me to               0.000894
hey hey hey hey              0.000873
Chandler Bing specific n-grams:
                    mean_tfidf_score
no no no

In [54]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer


n_gram = 3

# Calculate tf-idf scores for all n-grams in the utterances
vectorizer = TfidfVectorizer(ngram_range=(n_gram, n_gram))
tfidf = vectorizer.fit(all_lines)


# Transform the dialogue for each character into a TF-IDF matrix
monica_tfidf = tfidf.transform(monica_lines)
ross_tfidf = tfidf.transform(ross_lines)
chandler_tfidf = tfidf.transform(chandler_lines)
joey_tfidf = tfidf.transform(joey_lines)
rachel_tfidf = tfidf.transform(rachel_lines)

# Get the feature names (i.e., the n-grams) from the TfidfVectorizer
feature_names = tfidf.get_feature_names_out()


# Define a function to get the top n-grams for each character
def get_top_ngrams(tfidf_matrix, character_name, n):
    # Convert the TF-IDF matrix to a dense array
    dense_matrix = tfidf_matrix.toarray()
    # Get the row index for the character in the TF-IDF matrix
    character_index = all_dialogue.index(character_name)
    # Get the TF-IDF scores for the character's dialogue
    character_scores = dense_matrix[character_index]
    # Get the indices of the top n scores
    top_indices = character_scores.argsort()[-n:][::-1]
    # Get the corresponding n-gram strings and scores
    top_ngrams = [(feature_names[i], character_scores[i]) for i in top_indices]
    return top_ngrams





[['get it hi' 'it hi tim' 'debra winger had' ... 'are you doing'
  'what are you' 'oh my god']]
